In [11]:
import mysql.connector
import json
import os

In [12]:
# Setup MySQL connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="rootr00t",
    database="spotify_data"  # Make sure to specify your database
)
cursor = mydb.cursor()

# Function to insert playlist data
def insert_playlist(playlist):
    insert_playlist_query = """
    INSERT INTO playlists (playlist_id, name, num_tracks, num_albums, num_followers, num_edits, duration_ms, num_artists)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    playlist_data = (playlist['pid'], playlist['name'], playlist['num_tracks'], playlist['num_albums'],
                     playlist['num_followers'], playlist['num_edits'], playlist['duration_ms'], playlist['num_artists'])
    cursor.execute(insert_playlist_query, playlist_data)
    return cursor.lastrowid

# Function to insert song data
def insert_song(track_uri, artist_name, track_name, duration_ms):
    insert_song_query = """
    INSERT INTO songs (track_uri, artist_name, track_name, duration_ms)
    VALUES (%s, %s, %s, %s) ON DUPLICATE KEY UPDATE track_uri = track_uri
    """
    cursor.execute(insert_song_query, (track_uri, artist_name, track_name, duration_ms))

# Function to insert playlist_song_map data
def insert_playlist_song_map(playlist_id, track_uri):
    insert_mapping_query = """
    INSERT INTO playlist_song_map (playlist_id, track_uri)
    VALUES (%s, %s)
    """
    cursor.execute(insert_mapping_query, (playlist_id, track_uri))

In [13]:
directory_path = './Test Playlist'

for file_name in os.listdir(directory_path):
    if file_name.endswith('.json'):
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'r') as file:
            data = json.load(file)
            for playlist in data['playlists']:
                # Insert playlist and retrieve its ID
                playlist_id = insert_playlist(playlist)

                # Iterate through tracks in the playlist
                for track in playlist['tracks']:
                    # Insert song data
                    insert_song(track['track_uri'], track['artist_name'], track['track_name'], track['duration_ms'])
                    # Insert mapping data
                    insert_playlist_song_map(playlist_id, track['track_uri'])

# Commit the transactions after all inserts
mydb.commit()

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`spotify_data`.`playlist_song_map`, CONSTRAINT `playlist_song_map_ibfk_1` FOREIGN KEY (`playlist_id`) REFERENCES `playlists` (`playlist_id`))

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.